In [1]:
!pip install country_converter

     |████████████████████████████████| 49 kB 5.5 MB/s 
  Created wheel for country-converter: filename=country_converter-0.7.3-py3-none-any.whl size=52304 sha256=a7f48ba6a11f477e7d14b7fcae81550509f518bf47a7d0948e47670130c9a3ae
  Stored in directory: /root/.cache/pip/wheels/2e/d9/03/664296e7e90b85b7e8ffca3af6cacdb299eec3929d62746505
Successfully built country-converter


In [2]:
!pip install reverse_geocode


     |████████████████████████████████| 1.6 MB 31.9 MB/s 
  Created wheel for reverse-geocode: filename=reverse_geocode-1.4.1-py3-none-any.whl size=1637840 sha256=a13b50e9f7e742e7800b1268c45eccdb1ace4aecd996446e3e25e9c0b5d7d7e5
  Stored in directory: /root/.cache/pip/wheels/34/fd/8c/204b58f530154e61f182bcf1813a53b3ad5844faaff3df51d6
Successfully built reverse-geocode


In [3]:
!pip install geopy

In [4]:
import requests 
import pandas as pd
import os
from datetime import datetime
import country_converter as cc
import reverse_geocode
from geopy.geocoders import Nominatim


In [5]:
def dataConnection():
    url= 'https://www.aishub.net/stations/export'
    
    #Establishing connection ...
    response = requests.get(url)

    if response.status_code != 200:
        raise Exception("Failed! to establish the connection")
   
    #Getting text document
    text =  response.text

    #Saving into csv file 
    with open("stationStatus.csv", 'w') as file:

      #Splitting 
      lns = text.split("\r\n")
      for item in lns:
        file.write(item.replace("=", ",") + os.linesep)
    
dataConnection()

In [6]:
# Data cleaning ....

In [7]:
df = pd.read_csv('stationStatus.csv')
df.head()

,ID,Country,Location,Latitude,Longitude,Last signal
0,2766,ao,Zaire,-7.0000,13.000,1632831301
1,3061,ar,Buenos Aires,-34.4328,-58.558,1636940101
2,2675,au,Sydney,-33.7726,151.293,1636940101
3,2234,au,Mt Arthur (N.Tasmania),-41.2700,147.280,1634799602
4,2999,au,Mount Hicks,-41.0373,145.742,1636940101


In [8]:
#set(df['Location'])

dfclean = df.copy()

In [9]:

dfclean['station_ID'] = dfclean['ID']

In [10]:
#Checking null values ..
dfclean.isnull().sum()

ID             0
Country        0
Location       0
Latitude       0
Longitude      0
Last signal    0
station_ID     0
dtype: int64

In [11]:
dfclean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 887 entries, 0 to 886
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           887 non-null    int64  
 1   Country      887 non-null    object 
 2   Location     887 non-null    object 
 3   Latitude     887 non-null    float64
 4   Longitude    887 non-null    float64
 5   Last signal  887 non-null    int64  
 6   station_ID   887 non-null    int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 48.6+ KB


In [12]:
# converting into alpha 2 to alpha 3
dfclean['country_code'] =  dfclean['Country'].apply(lambda x :  cc.convert(names=x, to='ISO3', not_found = 'unknown')   )

zz not found in ISO2
zz not found in ISO2
zz not found in ISO2
zz not found in ISO2
zz not found in ISO2
zz not found in ISO2
zz not found in ISO2
zz not found in ISO2
zz not found in ISO2
zz not found in ISO2


In [13]:
#converting into country name
dfclean['country_name'] = dfclean['Country'].apply(lambda x: cc.convert(names = x , src='ISO2', to='name_short',not_found = 'unknown'))

zz not found in ISO2
zz not found in ISO2
zz not found in ISO2
zz not found in ISO2
zz not found in ISO2
zz not found in ISO2
zz not found in ISO2
zz not found in ISO2
zz not found in ISO2
zz not found in ISO2


In [14]:
dfclean.head()

,ID,Country,Location,Latitude,Longitude,Last signal,station_ID,country_code,country_name
0,2766,ao,Zaire,-7.0000,13.000,1632831301,2766,AGO,Angola
1,3061,ar,Buenos Aires,-34.4328,-58.558,1636940101,3061,ARG,Argentina
2,2675,au,Sydney,-33.7726,151.293,1636940101,2675,AUS,Australia
3,2234,au,Mt Arthur (N.Tasmania),-41.2700,147.280,1634799602,2234,AUS,Australia
4,2999,au,Mount Hicks,-41.0373,145.742,1636940101,2999,AUS,Australia


In [15]:
# Region name filter
dfclean['region_name'] = dfclean['Location'].apply(lambda x : (x.split()[0] +' ' + x.split()[1]) if len(x.split())>2 else x)

In [16]:
#findind location for zero latitude and longitude
unknown_location = dfclean.loc[dfclean['Latitude']==0.00 ]
unknown_location

,ID,Country,Location,Latitude,Longitude,Last signal,station_ID,country_code,country_name,region_name
73,2083,cn,Yangpu,0.0,0.0,1636940101,2083,CHN,China,Yangpu
74,2193,cn,Xia Men,0.0,0.0,1636940101,2193,CHN,China,Xia Men
75,3031,cn,Jiangyin & Maanshan,0.0,0.0,1636940101,3031,CHN,China,Jiangyin &
76,2044,cn,Fujian,0.0,0.0,1636940101,2044,CHN,China,Fujian
77,2598,cn,Beijing,0.0,0.0,1636940101,2598,CHN,China,Beijing
78,2587,cn,Beijing,0.0,0.0,1636940101,2587,CHN,China,Beijing
79,2113,cn,Xiawan,0.0,0.0,1630867201,2113,CHN,China,Xiawan
80,2930,cn,Chongqing,0.0,0.0,1636940101,2930,CHN,China,Chongqing
81,3394,cn,Chongqing,0.0,0.0,1636940102,3394,CHN,China,Chongqing
82,2456,cn,Haikou,0.0,0.0,1636437602,2456,CHN,China,Haikou


In [17]:
def fillingZero(dfclean)->pd.DataFrame:
  """Filling zero lat and long with city cordinate  """
  city_list = []
  for i in unknown_location['Location']:
    if i == 'At Sea':
      continue
    else:
      city_list.append(i)

  for i in city_list:
    try:
      index = dfclean[dfclean['Location']== i].index.values
      #extracting latitude
      lat = Nominatim(user_agent="Tausif").geocode(i , timeout=20).latitude
      #replacing in rows having zero cordinate
      dfclean.at[index,'Latitude']=lat

      #extracting longitude
      lng = Nominatim(user_agent="Tausif").geocode(i , timeout=20).longitude
      #replacing in rows having zero cordinate
      dfclean.at[index,'Longitude'] = lng
      
    except Exception:
      pass
  return dfclean

dfclean = fillingZero(dfclean)

In [18]:
def regionCode(dfclean)->pd.DataFrame:
  """ Extracting region code """
  region = []
  for x,y in zip(dfclean['region_name'] ,dfclean['country_code']) :

    if len(x.split())>1:
      x = x.split()[0:2]
      x = x[0][0] + x[1][0] + ' , ' + y
      region.append(x)
    else:
      x = x[0:2].upper() + ' , ' + y
      region.append(x)
  dfclean['region_code'] = pd.DataFrame(region)
  return dfclean
dfclean = regionCode(dfclean)

In [19]:
unknown_location= dfclean.loc[dfclean['Latitude']==0.00 ]
unknown_location

,ID,Country,Location,Latitude,Longitude,Last signal,station_ID,country_code,country_name,region_name,region_code
75,3031,cn,Jiangyin & Maanshan,0.0,0.0,1636940101,3031,CHN,China,Jiangyin &,"J& , CHN"
180,3433,de,At Sea,0.0,0.0,1636940102,3433,DEU,Germany,At Sea,"AS , DEU"
223,3022,de,At Sea,0.0,0.0,1636940101,3022,DEU,Germany,At Sea,"AS , DEU"
276,2670,in,At Sea,0.0,0.0,1636871702,2670,IND,India,At Sea,"AS , IND"
874,2376,zz,At Sea,0.0,0.0,1636940101,2376,unknown,unknown,At Sea,"AS , unknown"
875,2819,zz,At Sea,0.0,0.0,1636940101,2819,unknown,unknown,At Sea,"AS , unknown"
876,2992,zz,At Sea,0.0,0.0,1636940101,2992,unknown,unknown,At Sea,"AS , unknown"
877,3059,zz,At Sea,0.0,0.0,1636940101,3059,unknown,unknown,At Sea,"AS , unknown"
878,2258,zz,At Sea,0.0,0.0,1636698601,2258,unknown,unknown,At Sea,"AS , unknown"
880,3120,zz,At Sea,0.0,0.0,1636940101,3120,unknown,unknown,At Sea,"AS , unknown"


In [20]:
index_name = unknown_location.index

dfclean.drop(index_name , inplace= True)

In [21]:
def nearestCity(dfclean)->pd.DataFrame:
  """ This function will find the nearest city 
      according to the latitude and longitude
  """
  lat = dfclean['Latitude']
  lng = dfclean['Longitude']
  nearest_city = []
  for x , y in zip(lat , lng):
    if x == 0.0 and y == 0.0:
      continue
    cordinates = (x , y),
    city = reverse_geocode.search(cordinates)
    city = city[0]['city']
    nearest_city.append(city)
  dfclean['nearest_city'] = pd.DataFrame(nearest_city)

nearestCity(dfclean)


In [33]:
def states(dfclean)->pd.DataFrame:

  lat = dfclean['Latitude']
  lng = dfclean['Longitude']
  city_list = []
  state_list = []
  pin_code_list = []
  district_list = []

  for x , y in zip(lat , lng):
    
    location =  Nominatim(user_agent="Tausif").reverse(str(x) + ","+ str(y),timeout=20)
    try:
      address = location.raw['address']

      city = address.get('city','' )
      city_list.append(city)

      district = address.get('state_district', '')
      district_list.append(district)
      
      state = address.get('state', '')
      state_list.append(state)

      pin_code = address.get('postcode','' )
      pin_code_list.append(pin_code)

    except Exception:
      pass
    

  dfclean['city'] = pd.DataFrame(city_list)

  dfclean['district'] = pd.DataFrame(district_list)

  dfclean['state'] = pd.DataFrame(state_list)

  dfclean['postcode'] = pd.DataFrame(pin_code_list)

states(dfclean)
 

In [34]:
loc = Nominatim(user_agent="Tausif").reverse(str(-34.4328) + ',' + str(-58.558))
# address = loc.raw['address']
# print(address.get('city'))
# print(address.get('state'))
# print(address.get('postcode' ))
print(loc.raw)

{'place_id': 121568774, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 91450466, 'lat': '-34.43306941376188', 'lon': '-58.55776257962037', 'display_name': 'Alvear, San Fernando, Partido de San Fernando, Buenos Aires, 1646, Argentina', 'address': {'road': 'Alvear', 'town': 'San Fernando', 'city': 'San Fernando', 'state_district': 'Partido de San Fernando', 'state': 'Buenos Aires', 'postcode': '1646', 'country': 'Argentina', 'country_code': 'ar'}, 'boundingbox': ['-34.4337765', '-34.4295597', '-58.5585714', '-58.5537529']}


In [35]:
dfclean.head()

,ID,Country,Location,Latitude,Longitude,Last signal,station_ID,country_code,country_name,region_name,region_code,nearest_city,city,district,state,postcode
0,2766,ao,Zaire,-7.0000,13.000,1632831301,2766,AGO,Angola,Zaire,"ZA , AGO",N'zeto,,,Zaire,
1,3061,ar,Buenos Aires,-34.4328,-58.558,1636940101,3061,ARG,Argentina,Buenos Aires,"BA , ARG",Tigre,San Fernando,Partido de San Fernando,Buenos Aires,1646
2,2675,au,Sydney,-33.7726,151.293,1636940101,2675,AUS,Australia,Sydney,"SY , AUS",Curl Curl,Sydney,,New South Wales,2096
3,2234,au,Mt Arthur (N.Tasmania),-41.2700,147.280,1634799602,2234,AUS,Australia,Mt Arthur,"MA , AUS",Mayfield,,,Tasmania,7268
4,2999,au,Mount Hicks,-41.0373,145.742,1636940101,2999,AUS,Australia,Mount Hicks,"MH , AUS",Wynyard,,,Tasmania,7325


In [ ]:
#finding unknown country code location
sea_location = dfclean.loc[dfclean['country_code'] == 'unknown']
sea_location

,ID,Country,Location,Latitude,Longitude,Last signal,station_ID,country_code,country_name,region_name,region_code,nearest_city
872,2656,zz,Cagliari,39.2185,9.13968,1636719002,2656,unknown,unknown,Cagliari,"CA , unknown",NaN


In [38]:
#Converting unix time into date time object
dfclean['last_signal'] = pd.to_datetime(dfclean['Last signal'],unit = 's')

In [ ]:
# Duration of signal from current time
dfclean['Total_time_since_last_signal'] = dfclean['last_signal'].apply(lambda x : datetime.now()- x)

In [39]:
dfclean.drop(['Country','Latitude','Longitude','ID','Location' , 'Last signal'],axis = 1 , inplace= True)

In [40]:
dfclean.head()

,station_ID,country_code,country_name,region_name,region_code,nearest_city,city,district,state,postcode,last_signal
0,2766,AGO,Angola,Zaire,"ZA , AGO",N'zeto,,,Zaire,,2021-09-28 12:15:01
1,3061,ARG,Argentina,Buenos Aires,"BA , ARG",Tigre,San Fernando,Partido de San Fernando,Buenos Aires,1646,2021-11-15 01:35:01
2,2675,AUS,Australia,Sydney,"SY , AUS",Curl Curl,Sydney,,New South Wales,2096,2021-11-15 01:35:01
3,2234,AUS,Australia,Mt Arthur,"MA , AUS",Mayfield,,,Tasmania,7268,2021-10-21 07:00:02
4,2999,AUS,Australia,Mount Hicks,"MH , AUS",Wynyard,,,Tasmania,7325,2021-11-15 01:35:01


BBB not found in ISO3


'BBB'

In [41]:
dfclean.to_csv('ais_data.csv')